In [0]:
import pyspark.pandas as ps
from datetime import datetime, timedelta
import time


def atualiza_dados():
    """
    Lê o arquivo com as notícias já armazenadas e atualiza com as novas notícias.
    As notícias repetidas são removidas, mantendo somente as que possuem data de publicação mais recente.
    """
    try:
        flag = dbutils.fs.head('/FileStore/tables/projeto/flag.txt')
        if flag == '1':
            df_news = ps.read_table('%s.tb_news' % 'HEALTHGEN')
            criterios_duplicidade = ['url']
            df_news.drop_duplicates(subset=criterios_duplicidade, keep='last', inplace=True)
            df_news.fillna('None', inplace=True)
            
            df_news.to_parquet('/FileStore/tables/projeto/news.parquet')
            print("Dados atualizados no arquivo /FileStore/tables/projeto/news.parquet")
            
            # df_news.to_table('HEALTHGEN.tb_news', 'parquet')
            # print("Dados atualizados na tabela HEALTHGEN.tb_news")

            dbutils.fs.rm('/FileStore/tables/projeto/flag.txt', True)
            time.sleep(2)
            dbutils.fs.put('/FileStore/tables/projeto/flag.txt', '0')

        else:
            print("Não há novos dados para atualizar")
    except Exception as e:
        print(f"Erro na atualização:\n{e}")

while True:
    agora = datetime.now() - timedelta(hours=3)
    print(f"Verificando novos dados... {agora.strftime('%d/%m/%Y %H:%M:%S')}")
    atualiza_dados()
    time.sleep(30)

Verificando novos dados... 04/10/2023 19:59:25
Dados atualizados no arquivo /FileStore/tables/projeto/news.parquet
Wrote 1 bytes.
Verificando novos dados... 04/10/2023 20:00:02
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:00:32
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:01:02
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:01:32
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:02:03
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:02:33
Não há novos dados para atualizar
Verificando novos dados... 04/10/2023 20:03:03
Erro na atualização:
An error occurred while calling o396.head.
: java.io.FileNotFoundException: /FileStore/tables/projeto/flag.txt
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.

In [0]:
%sql
-- Comando para removar cache
REFRESH TABLE HEALTHGEN.tb_news